<a href="https://colab.research.google.com/github/ChathurangaChandrasekara/trafficControlSystem/blob/main/TrafficContrlSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pathlib
import cv2


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2301, done.
remote: Counting objects: 100% (2301/2301), done.
remote: Compressing objects: 100% (1999/1999), done.
remote: Total 2301 (delta 560), reused 931 (delta 279), pack-reused 0
Receiving objects: 100% (2301/2301), 30.60 MiB | 10.53 MiB/s, done.
Resolving deltas: 100% (560/560), done.


In [15]:
path = os.getcwd()
print(path)

/content


In [25]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import pathlib

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

ModuleNotFoundError: ignored

In [ ]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

In [ ]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.compat.v2.saved_model.load(str(model_dir), None)
  model = model.signatures['serving_default']

  return model

In [ ]:
path = os.getcwd()
print(path)

In [ ]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [ ]:
model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
detection_model = load_model(model_name)

In [ ]:
print(detection_model.inputs)

In [ ]:
detection_model.output_dtypes

In [ ]:
detection_model.output_shapes

In [ ]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  output_dict = model(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [ ]:
def show_inference(model, image_path):
    
#   image_np = np.array(Image.open(image_path))
  image_np = image_path

  image_np=cv2.cvtColor(image_np,cv2.COLOR_BGR2RGB)

  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)

  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=2,
      skip_labels=True)

#   image_np=cv2.cvtColor(image_np,cv2.COLOR_BGR2RGB)
#   display(Image.fromarray(image_np))
    
  output_list = get_count(output_dict['detection_boxes'],output_dict['detection_scores'],output_dict['detection_classes'])
  output_length = get_length(image_np,output_list[0],output_list[1],output_list[2])
  print("Detected object count :",output_list[3])
  print("Pixel value of queue length :",output_length[0])

  display(Image.fromarray(image_np))
    
  return output_list[3],output_length[0],image_np
 
#   return output_list[3],image_np


In [ ]:
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('test-image/new-image')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*g")))
TEST_IMAGE_PATHS


In [ ]:
import os
d = "test-image/new-image"
for path in os.listdir(d):
    full_path = os.path.join(d, path)
    if os.path.isfile(full_path):
        img = cv2.imread(full_path)
        img = show_inference(detection_model, img)


In [ ]:
for image_path in TEST_IMAGE_PATHS:
    print(image_path)
    
#     print(img)
#     img = cv2.imread(image_path)
#     img = show_inference(detection_model, img)
#     print(img)
#     if img!=None:
#         window_name = 'image'
#         cv2.imshow(window_name,img)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()

In [ ]:
# def get_vehicle(classes,scores):
#     vehicle_classes = [2,3,4,6,8]
#     c = 0
#     class_array = []
#     score_array = []
# #     print('classes = ',classes)
# #     print('lenght = ', len(classes))
# #     print('scores = ',scores)
# #     print('lenght = ', len(scores))
#     for i in classes:
#         for x in vehicle_classes:
#             if i==x:
#                 class_array.append(i)
#                 score_array.append(scores[c])
#         c += 1
    
#     print('class_array = ',class_array)
#     print('lenght = ', len(class_array))
#     print('score_array = ',score_array)
#     print('lenght = ', len(score_array))
#     return class_array,score_array

In [ ]:
def get_count(boxes,scores,classes):  
    count = 0
    vehicle_classes = [2,3,4,6,8]
    class_array = []
    score_array = []
    box_array = []
#     category_index = { 2: {'id': 2, 'name': 'bicycle'}, 3: {'id': 3, 'name': 'car'}, 4: {'id': 4, 'name': 'motorcycle'}, 6: {'id': 6, 'name': 'bus'}, 8: {'id': 8, 'name': 'truck'}}
    output = []
    
    for i in range(100):
        if (scores[i] > 0.5):
            for x in vehicle_classes:
                if(classes[i] == x):
                    class_array.append(x)
                    score_array.append(scores[i])
                    box_array.append(boxes[i])
                    
    x = len(class_array)
    output.append(class_array)
    output.append(score_array)
    output.append(box_array)
    output.append(x)
    return output

In [ ]:
def get_length(image,classes,score,boxes):
 MinY,MaxY,X,Length = 2,0,0.1,0
 c = 0
 for i in classes:
        h,w=image.shape[0:2]
    #   image.shape=[height,width,3]
        ymin,xmin,ymax,xmax=boxes[c]
        c += 1
        if MaxY<ymax:
            MaxY = ymax
#             X = xmax
        if MinY>ymin:
            MinY = ymin

            
 start_point = (int(round(X*w)),int(round(MinY*h))) 
 end_point = (int(round(X*w)), int(round(MaxY*h)) )
 color = (255,0,0) 
 thickness = 3
 Length = MaxY-MinY
    # Using cv2.line() method 
    # Draw a diagonal green line with thickness of 5 px 
 Image = cv2.line(image,start_point,end_point,color,thickness) 
 return Length,Image


In [ ]:
# PATH_TO_TEST_IMAGES_DIR = pathlib.Path('test-image')
# TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*g")))
# TEST_IMAGE_PATHS

In [ ]:
# for image_path in TEST_IMAGE_PATHS:
#     show_inference(detection_model, image_path)
    

In [ ]:
# import cv2

# img = cv2.imread('test-image\A4.jpg')
# # print(img)
# img=show_inference(detection_model,img)
# cv2.imshow('IMG',img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


In [ ]:
# import glob 
# img_dir = r"test-image"
# data_path = os.path.join(img_dir,'*g')
# files = glob.glob(data_path) 

# for file in files:
#     img=show_inference(detection_model,file)
#     cv2.imshow('IMG',file)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

In [ ]:
# print(files)

In [ ]:
import time
Time = 0
t = time.localtime()
current_time = time.strftime("%H:%M", t)
# current_time = "6:45"
t1 = "6:30"
t2 = "8:30"
t3 = "12:30"
t4 = "14:30"
t5 = "17:00"
t6 = "19:00"

print(current_time)

if(t1<current_time<t2):
    Time = 1
    
elif(t3<current_time<t4):
    Time = 3
    
elif(t5<current_time<t6):
    Time = 4
else:
    Time = 2
    
    
print(Time)

In [ ]:
import cv2
import os
def detect_image():
    c = 1
    length = []
    outputs = []
    outputs.append(Time)
    d = "test-image"
    for path in os.listdir(d):
        full_path = os.path.join(d, path)
        if os.path.isfile(full_path):
            img = cv2.imread(full_path)
            img = show_inference(detection_model, img)
#             print('count A',c,' = ' ,img[0])
#             print('pixel length A',c,' = ',img[1],'\n')
            L = round(img[1]*40)
            outputs.append(img[0])
            length.append(L)
            c += 1
    for i in length:
        outputs.append(i)
        
    return outputs
#         cv2.imshow('IMG',img[2])
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()


In [ ]:
# def crop_objects(image, image_np, output_dict, i):
#     global ymin, ymax, xmin, xmax
#     width, height = image.size

#     #Coordinates of detected objects
#     ymin = int(output_dict['detection_boxes'][0][0]*height)
#     xmin = int(output_dict['detection_boxes'][0][1]*width)
#     ymax = int(output_dict['detection_boxes'][0][2]*height)
#     xmax = int(output_dict['detection_boxes'][0][3]*width)
#     crop_img = image_np[ymin:ymax, xmin:xmax]

#     # 1. Only crop objects that are detected with an accuracy above 50%, 
#     # images 
#     # with objects below 50% will be filled with zeros (black image)
#     # This is something I need in my program
#     # 2. Only crop the object with the highest score (Object Zero)
#     if output_dict['detection_scores'][0] < 0.5:
#         crop_img.fill(0)

#     #Save cropped object into image
#     cv2.imwrite('Images/Step_2/' + str(i) + '.png', crop_img)
#     return ymin, ymax, xmin, xmax

In [ ]:
# import cv2

# video = cv2.VideoCapture(r'D:/Tensorflow/sherbrooke_video.avi')

# while(True):
    
#     ret,img=video.read()
#     img=show_inference(detection_model,img)
#     k=cv2.waitKey(1)
#     cv2.imshow('LIVE',img)
#     if(k==27):
#         break
# cv2.destroyAllWindows()

In [ ]:
# from datetime import datetime

# def bicycle_detect(image,classes,score,boxes):
    
#     for i in range(100):
#         if(classes[i]==4 and score[i]>0.8):
            
#             h,w=image.shape[0:2]
# #             image.shape=[height,width,3]
            
#             ymin,xmin,ymax,xmax=boxes[i]

# #             now = datetime.now()
# #             dt_string = now.strftime("%d_%m_%Y_%H_%M_%S")
            
#             center=(int(((xmin+xmax)/2)*w),int(((ymin+ymax)/2)*h))
#             cv2.circle(image,center,10,(0,0,255),-1)
            
#             file_name=os.path.join('E:/TEST/',dt_string+'.jpg')
#             cv2.imwrite(file_name,image)

In [ ]:
from tensorflow import keras
inputs = detect_image()
# print(image_detection)
#Set Headship lane respectivly A1,A2,A3,A4
headship = [1,1,0,0] 
for i in headship:
    inputs.append(i)
#Set Institutions respectivly A1,A2,A3,A4
institutions = [1,0,1,0]
for i in institutions:
    inputs.append(i)
    
# print(inputs)

In [ ]:
detection_model = keras.models.load_model(r'model_trained\40-64model.h5')

In [ ]:
detection_model.inputs

In [ ]:
detection_model.outputs

In [ ]:
import pandas as pd
dataset = np.array([inputs])
print(dataset.shape)
print(dataset[0].shape)
print(dataset[0:1].shape)

In [ ]:
prediction = detection_model.predict(dataset[0:1])

In [ ]:
# print(prediction)
result = np.argmax(prediction, axis=1)
array_num = 0
if(result==0):
    print('Lane A1')
    array_num = 1
    
elif(result==1):
    print('Lane A2')
    array_num = 2
    
elif(result==2):
    print('Lane A3')
    array_num = 3
    
elif(result==3):
    print('Lane A4')
    array_num = 4

print('Time limit for green signal :',inputs[array_num]*7)